<a href="https://colab.research.google.com/github/jinseriouspark/pricing/blob/main/%5Bdynamic_pricing%5D_04_multi_armed_bandit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamic pricing - 04. Multi Armed Bandit


- 의미 : dynamic pricing 에서는 멀티 암드 밴딧 게임을 사용해서 상품이나 서비스의 가격을 정할때, 어떤 가격이 가장 잘 팔리는지 알아보려고 한다. 여러가지 가격을 시도해보면서, 어떤 가격이 사람들에게 가장 합리적인지 알아내고, 해당 가격을 적용하고자 함


- 방법
  1. 탐험 exploration : 각각의 가격대를 몇번씩 시도하며 유저들의 반응을 살펴봄 (각각의 매출 분포를 이해)
  2. 활용 Exploitation : 어느정도 탐험을 하고난 다음, 유저들의 반응이 가장 높았던 조건의 가격대를 선정하고자 함

- 적용:  여러가지 가격을 시도하여 각 가격에서의 판매량(보상)을 관찰하고, 최적의 가격을 찾아내는 데 사용

## 멀티 암드 밴딧 알고리즘

### epsilon-greedy algorithm
- 미래를 생각하지 않고 각 단계에서 가장 최선의 선택을 하는 기법. 각 단계에서 최선의 선택을 한 것이 전체적으로도 최선이라는 것을 의미함

- e 의 확률로 탐색을 하고, 1-e 의 확률로 현재까지 가장 좋은 선택을 활용함

In [ ]:
import numpy as np


class EpsilonGreedyBandit:
  def __init__(self, k, epsilon, inital_value =0):
    self.k = k # 선택지의 개수
    self.epsilon = epsilon
    self.q_values = np.full(k, initial_value, dtype=float) # 각 선택의 예상 보상
    self.action_counts = np.zeros(k, dtype = int) # 각 선택지의 선택 횟수

  def select_action(self):
    if np.random.rand() < self.epsilon:
      return np.random.randint(self.k) # 탐색 : epsilon보다 작으면 탐색
    else:
      return np.argmax(self.q_values) # 활용 : 예상 보상이 가장 높은 선택지 채택